# Import Libraries

In [1]:
import numpy as np #numerical computation
import pandas as pd #data wrangling
import matplotlib.pyplot as plt #plotting package

# Import Dataset 

In [2]:
df = pd.read_csv('clean_2_train.csv')
df.head()

,Item_Identifier,Item_Weight,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_1,...,Item_Type_Combined_2,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,FDA15,9.30,0.922960,Dairy,249.8092,OUT049,1999,3735.1380,14,0,...,0,0,0,0,0,0,0,0,0,1
1,DRC01,5.92,1.003057,Soft Drinks,48.2692,OUT018,2009,443.4228,4,0,...,0,0,0,1,0,0,0,0,0,0
2,FDN15,17.50,0.831990,Meat,141.6180,OUT049,1999,2097.2700,14,0,...,0,0,0,0,0,0,0,0,0,1
3,FDX07,19.20,0.750000,Fruits and Vegetables,182.0950,OUT010,1998,732.3800,15,0,...,0,0,0,0,0,0,0,0,0,0
4,NCD19,8.93,0.666667,Household,53.8614,OUT013,1987,994.7052,26,1,...,1,1,0,0,0,0,0,0,0,0


#### Created a list remove_cols to remove those columns which doesn't required for Model Building 

In [3]:
remove_cols = [
    'Item_Identifier',
    'Item_Type',
    'Outlet_Identifier',
    'Outlet_Establishment_Year'
]
df = df.drop(remove_cols,axis =1)
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales,Outlet_Years,Item_Fat_Content_1,Item_Fat_Content_2,Outlet_Location_Type_1,Outlet_Location_Type_2,Outlet_Size_1,...,Item_Type_Combined_2,Outlet_1,Outlet_2,Outlet_3,Outlet_4,Outlet_5,Outlet_6,Outlet_7,Outlet_8,Outlet_9
0,9.30,0.922960,249.8092,3735.1380,14,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
1,5.92,1.003057,48.2692,443.4228,4,0,1,0,1,1,...,0,0,0,1,0,0,0,0,0,0
2,17.50,0.831990,141.6180,2097.2700,14,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
3,19.20,0.750000,182.0950,732.3800,15,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,8.93,0.666667,53.8614,994.7052,26,1,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0


In [4]:
df.shape

(8519, 25)

#### Data Preprocessing 

In [5]:
y = df.Item_Outlet_Sales.values
X = df.drop('Item_Outlet_Sales',axis = 1)

In [6]:
print(X.shape,y.shape)

(8519, 24) (8519,)


### Trained a Random Forest model

In [7]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()

#### List of parameters for hyperparameter tuning

In [8]:
param = {
    'max_depth':[3,6,9,12],
    'n_estimators' : [10,50,100,200] 
}

# Hyperparameter optimization using RandomizedSearchCV 

In [9]:
from sklearn.metrics import mean_squared_error,make_scorer
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(rf,param_distributions=param,n_iter=5,scoring=make_scorer(mean_squared_error),n_jobs=-1,cv=5,verbose=3)

In [10]:
random_search.fit(X,y)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed:   11.9s remaining:    3.7s
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed:   14.2s finished


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=5, n_jobs=-1,
                   param_distributions={'max_depth': [3, 6, 9, 12],
                                        'n_estimators': [10, 50, 100, 200]},
                   scoring=make_scorer(mean_squared_error), verbose=3)

In [11]:
means = random_search.cv_results_['mean_test_score']
params = random_search.cv_results_['params']
for mean, param in zip(means, params):
    print("%f with: %r" % (mean, param))
    if mean == min(means):
        print('Best parameters with the minimum Mean Square Error are:',param)

1212171.965558 with: {'n_estimators': 50, 'max_depth': 9}
1203943.597910 with: {'n_estimators': 100, 'max_depth': 9}
Best parameters with the minimum Mean Square Error are: {'n_estimators': 100, 'max_depth': 9}
1242554.264999 with: {'n_estimators': 50, 'max_depth': 12}
1387927.715629 with: {'n_estimators': 200, 'max_depth': 3}
1204071.376814 with: {'n_estimators': 10, 'max_depth': 6}


# Evaluating the model for Train and Test set 

In [12]:
rf = RandomForestRegressor(
    n_estimators=100, max_depth=6,
)

In [13]:
rf.fit(X,y)

RandomForestRegressor(max_depth=6)

In [14]:
y_pred = rf.predict(X)

In [17]:
from sklearn.metrics import r2_score,mean_squared_error

score = r2_score(y,y_pred)
print("Score:",100*score)
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y,y_pred)))

Score: 61.53361077164543
RMSE : 1058


In [18]:
from sklearn.metrics import mean_squared_error,make_scorer,mean_absolute_error
from sklearn.model_selection import cross_val_score

#Perform cross-validation:
cv_score = cross_val_score(rf,X, y, cv=20, scoring = make_scorer(mean_squared_error))
cv_score = np.sqrt(np.abs(cv_score))
    
#Print model report:
print("\nModel Report")
print("MAE : %.4g" % np.sqrt(mean_absolute_error(y,y_pred)))
print("RMSE : %.4g" % np.sqrt(mean_squared_error(y,y_pred)))
print("CV Score : Mean - %.4g | Std - %.4g | Min - %.4g | Max - %.4g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))


Model Report
MAE : 27.3
RMSE : 1058
CV Score : Mean - 1090 | Std - 47.1 | Min - 1021 | Max - 1210
